In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## This is the test file to generate 10 samples (description + code) in JSON format with different models.

## A. Generate project descriptions and titles with three different models

### 1. Setup the GPT-5 model and OpenRouter API key

In [2]:
!pip -q install requests jsonschema

In [3]:
import os, json, re, textwrap, datetime
import requests
from jsonschema import Draft7Validator

OPENROUTER_API_KEY = "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"
MODELS = [
    #"openai/gpt-5",
    "anthropic/claude-sonnet-4.5",
    "qwen/qwen3-coder",
]

### 2. Set the variables

In [4]:
SYSTEM_PROMPT = """
You are a meticulous AI project designer.
Your job: produce concise, implementable AI mini-project ideas.

Rules:
- Output must be valid JSON ONLY (no extra text).
- Each item has exactly two keys: "title" and "description".
- Titles are short and specific (≤ 6 words).
- Descriptions are 1–2 sentences, concrete, and implementable offline in 20–60 minutes.
- Prefer single-file, single-metric projects with tiny data and fast runtime.
- Avoid duplicate or near-duplicate ideas.
- Prefer standard Python libs or widely used ML libs (numpy, pandas, scikit-learn, PyTorch, TensorFlow, OpenCV).
- No external downloads; use built-in toy datasets (e.g., sklearn iris/digits) or tiny synthetic data.
""".strip()


In [5]:
FEW_SHOTS = """
{"title":"Iris SVM CLI",
 "description":"Train a small SVM on sklearn's iris dataset and print test accuracy. Provide a --predict flag that accepts four comma-separated features and prints the predicted class."}
{"title":"Canny Edge CLI",
 "description":"Load an image path or generate a 128×128 synthetic image, run OpenCV Canny with --t1/--t2 thresholds, save edges.png, and print TEST_PASS if the edge map is non-empty."}
""".strip()


### 3. A function for describing the task

In [6]:
import textwrap

def build_task(n=10):
    return textwrap.dedent(f"""
    Task: Generate {n} distinct AI mini-project ideas.

    Constraints:
    - Return a JSON array of length {n}.
    - Each item: object with exactly "title" (string) and "description" (string).
    - No comments, no prose outside JSON.

    Scope & Simplicity:
    - Each project is doable offline in 20–60 minutes on CPU.
    - Single-file mindset: one clear goal, one primary metric or artifact.
    - Keep dependencies minimal (numpy/pandas/sklearn/torch/tf/opencv only).

    Data:
    - Do NOT require internet downloads.
    - Use built-in toy datasets (e.g., sklearn iris/digits) or tiny synthetic data generation.
    - If an image/text input is needed, allow a synthetic fallback.

    Focus areas ONLY:
    - NLP, CV, Tabular. Do not include TimeSeries or Audio.

    Description style:
    - Titles ≤ 6 words, specific.
    - Descriptions are 1–2 sentences with concrete I/O hints (flags, paths, outputs).
    - Include at least one quick validation (e.g., accuracy threshold, file existence, non-empty output).

    Diversity (within the allowed areas):
    - Avoid repeating the same idea or trivial variants.

    Follow the style of these examples without repeating them:
    {FEW_SHOTS}

    Now produce the JSON array of {n} items.
    """).strip()


### 4. OpenRouter call helper

In [7]:
# Helper to make a safe Python variable name from a slug
def varname_from_slug(slug: str) -> str:
    name = slug.lower().replace("/", "_").replace("-", "_").replace(".", "_")
    return f"{name}_result"

In [8]:
# Generic OpenRouter caller taking model_id
def call_openrouter_model(model_id, messages, temperature=0.2, top_p=0.9, max_tokens=6000):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://colab.research.google.com/",
        "X-Title": f"Multi-Model Project Generator",
    }
    payload = {
        "model": model_id,
        "messages": messages,
        "temperature": float(temperature),
        "top_p": float(top_p),
        "max_tokens": int(max_tokens),
    }
    t0 = time.time()
    r = requests.post(url, headers=headers, json=payload, timeout=120)
    latency = time.time() - t0
    r.raise_for_status()
    content = r.json()["choices"][0]["message"]["content"]
    return content, latency

In [9]:
# Validator for title/description array
from jsonschema import Draft7Validator
def validate_items(arr, N):
    ITEM_SCHEMA = {
        "type":"object",
        "required":["title","description"],
        "properties":{
            "title":{"type":"string","minLength":3, "maxLength":100},
            "description":{"type":"string","minLength":20, "maxLength":600}
        },
        "additionalProperties": False
    }
    ARRAY_SCHEMA = {"type":"array","items":ITEM_SCHEMA, "minItems":N, "maxItems":N}
    errs = [e.message for e in Draft7Validator(ARRAY_SCHEMA).iter_errors(arr)]
    titles = [ (x.get("title") or "").strip().lower() for x in arr ]
    if len(set(titles)) != len(titles):
        errs.append("Duplicate titles detected.")
    return errs

In [10]:
# Robust extractor (keeps your previous logic)
import re, json, time, requests
def extract_json_array(text: str):
    m = re.search(r"```(?:json)?\s*([\s\S]*?)\s*```", text, re.IGNORECASE)
    if m:
        text = m.group(1).strip()
    start = text.find('[')
    if start == -1:
        return None
    depth = 0
    for i in range(start, len(text)):
        ch = text[i]
        if ch == '[': depth += 1
        elif ch == ']':
            depth -= 1
            if depth == 0:
                return text[start:i+1]
    return None

In [11]:
# Build a single shared message set
N = 10
task = build_task(N) + "\n\nReturn a raw JSON array only — no prose, no code fences, no markdown."
messages = [
    {"role":"system","content": SYSTEM_PROMPT},
    {"role":"user","content": task}
]

# Loop models and create a separate variable per model with the results
for model_id in MODELS:
    print(f"\n===== {model_id} =====")
    varname = varname_from_slug(model_id)         # e.g. "openai_gpt_5_result"
    try:
        raw, secs = call_openrouter_model(model_id, messages, temperature=0.2)
        json_str = extract_json_array(raw)
        if not json_str:
            globals()[varname] = {
                "raw": raw, "json_str": None, "items": None,
                "errors": ["No JSON array found"], "latency": secs
            }
            print(f"❌ No JSON array found | {secs:.1f}s")
            continue

        items = json.loads(json_str)
        errors = validate_items(items, N)

        globals()[varname] = {
            "raw": raw, "json_str": json_str, "items": items,
            "errors": errors, "latency": secs
        }

        if errors:
            print(f"⚠️ Parsed but validation errors ({len(errors)}) | {secs:.1f}s")
            for e in errors[:5]:
                print(" -", e)
        else:
            print(f"✅ Valid JSON ({len(items)} items) | {secs:.1f}s")
            print(json.dumps(items[:2], indent=2, ensure_ascii=False))  # preview first 2
    except Exception as e:
        globals()[varname] = {
            "raw": None, "json_str": None, "items": None,
            "errors": [str(e)], "latency": None
        }
        print("❌ Exception:", e)


===== anthropic/claude-sonnet-4.5 =====
✅ Valid JSON (10 items) | 15.0s
[
  {
    "title": "Digits KNN Classifier",
    "description": "Train a k-NN classifier on sklearn's digits dataset and report test accuracy. Accept --k flag to set neighbors and --predict flag with a 64-element comma-separated list to classify a single digit."
  },
  {
    "title": "Synthetic Text Sentiment Analyzer",
    "description": "Generate 200 synthetic sentences with positive/negative labels using keyword rules, train a logistic regression on TF-IDF features, and print validation accuracy. Accept --text flag to classify a new sentence."
  }
]

===== qwen/qwen3-coder =====
✅ Valid JSON (10 items) | 15.8s
[
  {
    "title": "Digit Classifier CLI",
    "description": "Train a logistic regression on sklearn's digits dataset and print test accuracy. Add a --predict flag to classify a comma-separated 64-element pixel vector from stdin."
  },
  {
    "title": "Synthetic Blob Clustering",
    "description": "Gene

In [12]:
for model_id in MODELS:
    print(" -", varname_from_slug(model_id))

 - anthropic_claude_sonnet_4_5_result
 - qwen_qwen3_coder_result


### All the resuts for three models that I used are below:

In [13]:
print(json.dumps(anthropic_claude_sonnet_4_5_result['items'], indent=2, ensure_ascii=False))

[
  {
    "title": "Digits KNN Classifier",
    "description": "Train a k-NN classifier on sklearn's digits dataset and report test accuracy. Accept --k flag to set neighbors and --predict flag with a 64-element comma-separated list to classify a single digit."
  },
  {
    "title": "Synthetic Text Sentiment Analyzer",
    "description": "Generate 200 synthetic sentences with positive/negative labels using keyword rules, train a logistic regression on TF-IDF features, and print validation accuracy. Accept --text flag to classify a new sentence."
  },
  {
    "title": "Gaussian Blob Clustering CLI",
    "description": "Generate 300 2D points from three Gaussian blobs, run k-means clustering, and save a scatter plot colored by cluster. Print silhouette score and TEST_PASS if score > 0.5."
  },
  {
    "title": "MNIST Subset Neural Net",
    "description": "Load 1000 samples from sklearn's digits dataset, train a small PyTorch MLP for 10 epochs, and print final test accuracy. Save model w

In [14]:
print(json.dumps(qwen_qwen3_coder_result['items'], indent=2, ensure_ascii=False))

[
  {
    "title": "Digit Classifier CLI",
    "description": "Train a logistic regression on sklearn's digits dataset and print test accuracy. Add a --predict flag to classify a comma-separated 64-element pixel vector from stdin."
  },
  {
    "title": "Synthetic Blob Clustering",
    "description": "Generate 2D isotropic Gaussian blobs, fit k-means with k=3, plot clusters with centroids, and save to clusters.png. Validate with inertia < 50."
  },
  {
    "title": "Iris Decision Tree Viz",
    "description": "Train a decision tree on iris dataset, export to iris_tree.dot, and print TEST_PASS if the dot file contains at least 5 nodes."
  },
  {
    "title": "MNIST CNN Scratch",
    "description": "Build a 2-layer CNN in PyTorch on MNIST (use torchvision.datasets.MNIST with download=False), train for 1 epoch, and print final train accuracy > 80%."
  },
  {
    "title": "Text Sentiment SVM",
    "description": "Use sklearn's movie_reviews corpus (or synthetic binary strings), train a TF-

## 2. Generating the codes for each projects that have been generated earlier with the models

In [15]:
# -------- External memory (compact, curated) --------
memory = {
  "style_guide": [
    "Single-file script with `if __name__ == '__main__':` entrypoint.",
    "Use argparse with clear --help and sensible defaults.",
    "Prefer standard library datasets (sklearn, torchvision, keras).",
    "Attempt auto-download/cache with short timeout; if unavailable, fallback to a tiny structured synthetic dataset.",
    "Fix randomness: set seeds for random, numpy; torch if used; run on CPU by default.",
    "Validate inputs (paths, columns, image loads) and fail gracefully with one-line reason.",
    "Keep runtime < 2 minutes (few epochs, small subsets).",
    "Print `TEST_PASS` on success; otherwise `TEST_FAIL: <reason>`."
  ],
  "lessons": [
    "When standardizing features use sklearn.pipeline.Pipeline to avoid leakage.",
    "For OpenCV Canny, expose --threshold1 and --threshold2; convert to grayscale before edges.",
    "For CSV tasks, explicitly validate required columns; show a friendly error if missing.",
    "For plotting, save figures to disk and plt.close() to avoid backend issues.",
    "One file only. Return exactly ONE ```python block. No extra prose.",
    "CLI + help. Use a single 'argparse.ArgumentParser()'. All help strings are single-line (no embedded newlines).",
    "Seeds in 'main()'. Expose '--seed' and set seeds for random, numpy, and torch (if present) inside main().",
    "Data access policy. Only use library datasets when --allow-download is passed. Otherwise do not download; use a robust fallback (sklearn tabular, torchvision FakeData, PIL shapes, etc.). If using 20NG, call with download_if_missing=False unless allowed.",
    "Task–dataset match. Choose datasets that match the task (e.g., do not use 20 Newsgroups for spam/ham).",
    "CV safety. For OpenCV: 1. convert to grayscale if needed. 2. ensure input to detectors is uint8 (cv2.convertScaleAbs if needed). 3. for Haar, check face_cascade.empty() == False or fail.",
    "Acceptance contract. Implement explicit pass/fail checks (files exist, metrics ≥ thresholds, non-empty edge map, etc.). Print TEST_PASS only when all conditions hold; otherwise TEST_FAIL: <reason> and sys.exit(1).",
    "No broken syntax. Never split identifiers across lines. Never break f-strings or string literals across lines.",
    "End marker. Append '# END_OF_SCRIPT' as the last line of the file."
  ],
  "snippets": [
    # seed block to embed in each script
    "import random, numpy as np\nrandom.seed(42)\nnp.random.seed(42)\ntry:\n    import torch\n    torch.manual_seed(42)\nexcept Exception:\n    pass"
  ]
}

In [16]:
SYSTEM_PROMPT_CODEGEN = """
You are a meticulous senior Python engineer who writes production-quality, runnable scripts.
Priorities: (1) correctness, (2) reproducibility, (3) clarity, (4) speed.

Formatting & Output Contract:
- Return ONE code block only: ```python ...```
- The code must be a single file with `if __name__ == "__main__":` entrypoint.
- Provide a clear CLI via argparse and `--help`. All help strings must be single-line (no embedded newlines).
- Do not print explanations. Do not include markdown outside the single code block.
- Append `# END_OF_SCRIPT` as the final line of the file.

Data Access Policy:
- Prefer standard, library-provided datasets that Colab typically caches:
  - sklearn.datasets (iris, wine, diabetes, digits)
  - torchvision.datasets (MNIST, FashionMNIST, CIFAR-10/100)
  - tensorflow.keras.datasets (mnist, cifar10/cifar100, imdb, reuters)
- Only attempt loading/auto-downloading these datasets when the user passes `--allow-download`.
- If `--allow-download` is not passed, do NOT download; immediately fall back to a robust local/synthetic option
  (e.g., sklearn tabular, torchvision FakeData, PIL-drawn shapes, sklearn digits).
- For sklearn 20 Newsgroups, call with `download_if_missing=False` unless `--allow-download` is passed.
- Use small subsets and few epochs to keep runtime < 2 minutes.
- Do not call arbitrary external URLs or third-party APIs.

Behavioral Rules:
- Expose `--seed` and set seeds **inside `main()`** for `random`, `numpy`, and `torch` (if available); run on CPU by default.
- Choose datasets that match the task semantics (e.g., do NOT use 20 Newsgroups for spam/ham).
- Validate inputs (paths, columns, image loads, audio length) and fail gracefully with a concise message.
- For OpenCV tasks: convert to grayscale when needed; ensure `uint8` input (use `cv2.convertScaleAbs` if necessary);
  for Haar cascades, ensure `face_cascade.empty() == False` or fail.
- Keep the code minimal, readable, and fully runnable in a fresh Colab.
- Never split identifiers across lines; never break string literals or f-strings across lines.
- Implement explicit acceptance checks tied to the task (files exist, metrics ≥ thresholds, non-empty edge map, etc.).
- Print `TEST_PASS` only when all acceptance conditions hold; otherwise print `TEST_FAIL: <reason>` and `sys.exit(1)`.

Self-Check Before Returning (silently revise if any item fails):
- argparse help strings are single-line.
- Seeds are applied in `main()` for random/numpy/torch.
- No downloads unless `--allow-download` is passed; otherwise robust fallback is used.
- Dataset matches task semantics.
- Acceptance checks implemented; `TEST_PASS`/`TEST_FAIL` behavior present.
- File ends with `# END_OF_SCRIPT`.
- Code parses without SyntaxError.
""".strip()


In [17]:
FEWSHOTS_CODE = """
Example A (tabular classification with sklearn iris -> fallback synthetic; seeds-in-main; single-line help; acceptance checks)
```python
import argparse, sys
import random, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

def load_iris_or_synthetic(seed=42):
    try:
        from sklearn.datasets import load_iris  # no download required
        data = load_iris()
        X, y, used = data.data, data.target, "iris"
    except Exception:
        rng = np.random.default_rng(seed)
        n = 210
        c = rng.integers(0, 3, size=n)
        X = rng.normal(0, 1, size=(n, 4)) + c[:, None] * 1.5
        y, used = c, "synthetic"
    return X, y, used

def main():
    p = argparse.ArgumentParser(description="Iris (no-download) or synthetic fallback; seeds set in main; explicit acceptance.")
    p.add_argument("--test-size", type=float, default=0.2, help="Test set fraction (default: 0.2).")
    p.add_argument("--seed", type=int, default=42, help="Random seed (default: 42).")
    args = p.parse_args()

    # seeds in main
    random.seed(args.seed); np.random.seed(args.seed)
    try:
        import torch; torch.manual_seed(args.seed)
    except Exception:
        pass

    X, y, used = load_iris_or_synthetic(args.seed)
    if X is None or y is None or len(X) == 0:
        print("TEST_FAIL: dataset not available"); sys.exit(1)

    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=args.test_size, random_state=args.seed, stratify=y)
    clf = Pipeline([("scaler", StandardScaler()), ("lr", LogisticRegression(max_iter=300))])
    clf.fit(Xtr, ytr)
    acc = clf.score(Xte, yte)
    print(f"dataset={used} accuracy={acc:.3f}")
    # acceptance: stricter if iris, looser if synthetic
    if acc >= (0.85 if used == "iris" else 0.70):
        print("TEST_PASS")
    else:
        print("TEST_FAIL: accuracy below threshold"); sys.exit(1)

if __name__ == "__main__":
    main()
# END_OF_SCRIPT
```

Example B (vision MNIST with opt-in download -> fallback FakeData; uint8 safety; seeds-in-main; acceptance checks)
```python
import argparse, sys, os
import random, numpy as np

def load_mnist_or_fakedata(max_train=2000, max_test=500, seed=42, allow_download=False):
    try:
        import torch
        from torchvision import datasets, transforms
        torch.manual_seed(seed)
        tfm = transforms.ToTensor()
        # only download if explicitly allowed
        train = datasets.MNIST(root="./data", train=True, download=bool(allow_download), transform=tfm)
        test  = datasets.MNIST(root="./data", train=False, download=bool(allow_download), transform=tfm)
        # if dataset objects are empty because cache missing and download disabled, trigger fallback
        if len(train) == 0 or len(test) == 0:
            raise RuntimeError("MNIST cache missing and download disabled")
        train = torch.utils.data.Subset(train, list(range(min(len(train), max_train))))
        test  = torch.utils.data.Subset(test,  list(range(min(len(test),  max_test))))
        return train, test, True
    except Exception:
        import torch
        from torchvision import transforms
        from torchvision.datasets import FakeData
        torch.manual_seed(seed)
        tfm = transforms.ToTensor()
        train = FakeData(size=max_train, image_size=(1, 28, 28), num_classes=10, transform=tfm)
        test  = FakeData(size=max_test,  image_size=(1, 28, 28), num_classes=10, transform=tfm)
        return train, test, False

def main():
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader

    p = argparse.ArgumentParser(description="MNIST (opt-in download) or FakeData fallback; seeds in main; explicit acceptance.")
    p.add_argument("--epochs", type=int, default=1, help="Training epochs (default: 1).")
    p.add_argument("--batch", type=int, default=128, help="Batch size (default: 128).")
    p.add_argument("--seed", type=int, default=42, help="Random seed (default: 42).")
    p.add_argument("--allow-download", action="store_true", help="Permit MNIST download if not cached.")
    args = p.parse_args()

    # seeds in main
    random.seed(args.seed); np.random.seed(args.seed); torch.manual_seed(args.seed)

    train_ds, test_ds, real = load_mnist_or_fakedata(seed=args.seed, allow_download=args.allow_download)
    train = DataLoader(train_ds, batch_size=args.batch, shuffle=True)
    test  = DataLoader(test_ds,  batch_size=args.batch, shuffle=False)

    class TinyCNN(nn.Module):
        def __init__(self):
            super().__init__()
            self.net = nn.Sequential(
                nn.Conv2d(1, 16, 3, 1), nn.ReLU(),
                nn.MaxPool2d(2),
                nn.Conv2d(16, 32, 3, 1), nn.ReLU(),
                nn.MaxPool2d(2),
                nn.Flatten(),
                nn.Linear(32*5*5, 64), nn.ReLU(),
                nn.Linear(64, 10)
            )
        def forward(self, x): return self.net(x)

    model = TinyCNN()
    opt = optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()

    model.train()
    for _ in range(args.epochs):
        for xb, yb in train:
            # ensure uint8 -> float32 is handled by ToTensor; just train
            opt.zero_grad()
            logits = model(xb)
            loss = loss_fn(logits, yb)
            loss.backward(); opt.step()

    # eval + acceptance
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for xb, yb in test:
            pred = model(xb).argmax(1)
            correct += (pred == yb).sum().item()
            total += yb.numel()
    acc = correct / max(total, 1)
    print(f"acc={acc:.3f} dataset={'mnist' if real else 'fake'}")
    # stricter if real, looser if fake
    if acc >= (0.85 if real else 0.20):
        print("TEST_PASS")
    else:
        print("TEST_FAIL: accuracy below threshold"); sys.exit(1)

if __name__ == "__main__":
    main()
# END_OF_SCRIPT
```
""".strip()

In [18]:
import os, re, json, requests

# --- helpers ---
def extract_code_block(text: str) -> str:
    """
    Return the first code block content if present; otherwise return the whole text.
    Prefers ```python ... ``` but accepts ``` ... ```.
    """
    m = re.search(r"```(?:python)?\s*([\s\S]*?)\s*```", text, re.IGNORECASE)
    return m.group(1) if m else text

def print_long(s: str, width: int = 4000):
    """Print long strings without Colab truncation, in chunks."""
    for i in range(0, len(s), width):
        print(s[i:i+width])

In [19]:
# -------- Task builder (per project) --------
def build_code_task(project, memory):
    guide = "\n- ".join(memory["style_guide"])
    lessons = "\n- ".join(memory["lessons"][-6:])
    seed_block = memory["snippets"][0]

    return f"""
    PROJECT TITLE:
    {project['title']}

    PROJECT DESCRIPTION:
    {project['description']}

    Follow this style guide:
    - {guide}

    Incorporate recent lessons:
    - {lessons}

    Hard guardrails (must follow):
    - Return ONE code block only: ```python ...``` (no extra prose).
    - Single file with `if __name__ == "__main__":` entrypoint.
    - Use argparse; **all help strings are single-line** (no embedded newlines).
    - Expose `--seed` and set seeds **inside `main()`** for random, numpy, and torch (if available).
    - **Data policy:** Only attempt library dataset downloads when `--allow-download` is passed. Otherwise DO NOT download; use a robust fallback suited to this task (e.g., sklearn tabular, torchvision FakeData, PIL-drawn shapes, sklearn digits).
    - If using sklearn 20 Newsgroups, call with `download_if_missing=False` unless `--allow-download` is passed.
    - Choose datasets that **match the task semantics** (e.g., do NOT use 20 Newsgroups for spam/ham).
    - For OpenCV tasks: convert to grayscale when needed; ensure `uint8` input (use `cv2.convertScaleAbs` if necessary); for Haar cascades verify `face_cascade.empty()==False` or fail.
    - Never split identifiers across lines; never break string literals or f-strings across lines.

    Embed this seed block near the top of the script:
    ```python
    {seed_block}"""


In [20]:
# -------- OpenRouter caller (model passed in) --------

def call_openrouter_model(model_id, messages, temperature=0.2, top_p=0.9, max_tokens=7000):
  url = "https://openrouter.ai/api/v1/chat/completions"
  headers = {
  "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  "Content-Type": "application/json",
  "HTTP-Referer": "https://colab.research.google.com/",
  "X-Title": "Code Generation (per-model)",
  }
  if not headers["Authorization"]:
    raise RuntimeError("OPENROUTER_API_KEY environment variable not set.")
  payload = {
    "model": model_id,
    "messages": messages,
    "temperature": float(temperature),
    "top_p": float(top_p),
    "max_tokens": int(max_tokens),
    }
  r = requests.post(url, headers=headers, json=payload, timeout=120)
  r.raise_for_status()
  return r.json()["choices"][0]["message"]["content"]

In [21]:
#-------- Map: model slug -> per-model dict variable name --------

MODEL_TO_RESULTVAR = {
#"openai/gpt-5": "openai_gpt_5_result",
"anthropic/claude-sonnet-4.5": "anthropic_claude_sonnet_4_5_result" ,
"qwen/qwen3-coder": "qwen_qwen3_coder_result"
}

In [22]:
# -------- Generate code for the first 2 projects per model --------
import time

print("Generating code for first 2 items of each model's projects...\n")
per_model_generated_code = {}

def code_items_varname(slug: str) -> str:
    return slug.lower().replace("/", "_").replace("-", "_").replace(".", "_") + "_code_items"


for model_id, varname in MODEL_TO_RESULTVAR.items():
    result = globals().get(varname)
    if not result or not result.get("items"):
        print(f"Skipping {model_id}: no items found in `{varname}`")
        continue

    projects = result["items"][:2]  # first 2 only for test
    print(f"\n===== {model_id}: generating and attaching code for {len(projects)} projects =====")

    items_with_code = []
    for idx, proj in enumerate(projects, 1):
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT_CODEGEN},
            {"role": "user", "content": "Study these two short code examples and copy their structure (CLI, dataset policy, seeds, TEST_PASS contract)."},
            {"role": "assistant", "content": FEWSHOTS_CODE},
            {"role": "user", "content": build_code_task(proj, memory)},
        ]
        raw = call_openrouter_model(model_id, messages, temperature=0.2, max_tokens=7000)
        code = extract_code_block(raw)
        time.sleep(10)

        item = {
            "title": proj["title"],
            "description": proj["description"],
            "code": code
        }
        items_with_code.append(item)

        # show the full code (no truncation)
        print(f"\n--- {model_id} • Project {idx}: {proj['title']} ---\n")
        print_long(code)  # full code printed

    # put per-model list into a dedicated variable
    var_codes = code_items_varname(model_id)  # e.g., openai_gpt_5_code_items
    globals()[var_codes] = items_with_code

    # also store inside the original result dict under 'items_with_code' for convenience
    result["items_with_code"] = items_with_code

    # pretty JSON view of the per-model list
    print(f"\n>>> {model_id} • JSON with title, description, code:")
    print_long(json.dumps(items_with_code, indent=2, ensure_ascii=False))

Generating code for first 2 items of each model's projects...


===== anthropic/claude-sonnet-4.5: generating and attaching code for 2 projects =====

--- anthropic/claude-sonnet-4.5 • Project 1: Digits KNN Classifier ---

import argparse
import sys
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def load_digits_or_synthetic(seed=42):
    """Load sklearn digits (no download required) or fallback to synthetic."""
    try:
        from sklearn.datasets import load_digits
        data = load_digits()
        X, y, used = data.data, data.target, "digits"
    except Exception:
        rng = np.random.default_rng(seed)
        n = 1797
        X = rng.uniform(0, 16, size=(n, 64))
        y = rng.integers(0, 10, size=n)
        used = "synthetic"
    return X, y, used

def main():
    p = argparse.ArgumentParser(description="Train k-NN on sklearn digits; predict

HTTPError: 429 Client Error: Too Many Requests for url: https://openrouter.ai/api/v1/chat/completions

In [23]:
print(json.dumps(anthropic_claude_sonnet_4_5_result["items_with_code"], indent=2, ensure_ascii=False))

[
  {
    "title": "Digits KNN Classifier",
    "description": "Train a k-NN classifier on sklearn's digits dataset and report test accuracy. Accept --k flag to set neighbors and --predict flag with a 64-element comma-separated list to classify a single digit.",
    "code": "import argparse\nimport sys\nimport random\nimport numpy as np\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.metrics import accuracy_score\n\ndef load_digits_or_synthetic(seed=42):\n    \"\"\"Load sklearn digits (no download required) or fallback to synthetic.\"\"\"\n    try:\n        from sklearn.datasets import load_digits\n        data = load_digits()\n        X, y, used = data.data, data.target, \"digits\"\n    except Exception:\n        rng = np.random.default_rng(seed)\n        n = 1797\n        X = rng.uniform(0, 16, size=(n, 64))\n        y = rng.integers(0, 10, size=n)\n        used = \"synthetic\"\n    return X, y, used\n\ndef main(

**Note:** So now we have more structured and more correct project codes and descriptions that before which they are acceptable for finetuning the smaller models.